In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import warnings
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')

In [9]:
# Load Mosaik data
csv_file = "output/coords_inside_box.csv"
data = pd.read_csv(csv_file)

In [10]:
# Columns with MOSAIKS features
mosaik_features = data.iloc[:, 6:-1]

# Transform each 4000 MOSAIKS values into a single value using PCA
pca = PCA(n_components=1)
data['PCA_1'] = pca.fit_transform(mosaik_features)

In [27]:
# Load the shapefile containing the village polygons
polygon_data = gpd.read_file("villages_shapefiles/villages_shapefiles.shp")

# Find all unique village IDs
v_shp_ids = data['v_shp_id'].unique()

# Ensure the 'v_shp_id' column is of type int for both dataframes
polygon_data['v_shp_id'] = polygon_data['v_shp_id'].astype(int)

In [49]:
# Convert all the data into heatmaps
for i in range(len(v_shp_ids)):
	selected_village = v_shp_ids[i]

	selected_data = data[data['v_shp_id'] == selected_village]
	selected_polygon = Polygon(polygon_data[polygon_data['v_shp_id'] == int(selected_village)].iloc[0]['geometry'])

	x_min, y_min, x_max, y_max = selected_polygon.bounds
	map_center = [(y_min + y_max) / 2, (x_min + x_max) / 2]

	m = folium.Map(location=map_center, zoom_start=15)
	sim_geo = gpd.GeoSeries(selected_polygon).simplify(tolerance=0.001)
	geo_j = sim_geo.to_json()
	geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillOpacity": "0%"})
	folium.Popup(i).add_to(geo_j)
	geo_j.add_to(m)

	# Uncomment to draw heatmaps for fine grain version (coords inside polygon)
	# heatmap_data = [[row['lat'], row['lon'], row['PCA_1']] for idx, row in selected_data.iterrows() if selected_polygon.contains(Point(row['lon'], row['lat']))]

	# Heatmap for coords inside box
	heatmap_data = [[row['lat'], row['lon'], row['PCA_1']] for idx, row in selected_data.iterrows()]
	HeatMap(heatmap_data).add_to(m)
	
	m.save(f"heatmaps_box/heatmap_{selected_village}.html")
	# m.save(f"heatmaps/heatmap_{selected_village}.html")